# Lab

In [1]:
import logging
from datetime import datetime
from scraper.constants import BatchSpan
from scraper.date import DateTime
logger = logging.getLogger(__name__)

In [2]:
import logging
import sys

# Configure logging to print to standard output
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Example log messages
logging.debug("This is a debug message.") # Won't appear with level=INFO
logging.info("This is an info message.")
logging.warning("This is a warning message.")


INFO:root:This is an info message.


In [3]:
cs = BatchSpan.from_value(value='d')
print(cs.value)
print(cs.name)
print(cs.fmt)


d
DAY
%Y-%m-%d


In [4]:
start = datetime(2025, 6, 22)
td = datetime.now() - start
DateTime.get_minutes(td=td)

442.0

## CircuitBreaker

In [5]:
from scraper.monitor import CircuitBreaker

In [6]:
batch = 0
submissions = 0
comments = 0
context = {}
cb = CircuitBreaker()
for i in range(6):
    batch += 1
    submissions += i*10
    comments += i*100
    context = {"Batch": batch, "Submissions": submissions, "Comments": comments}
    cb.failure(context=context)

INFO:root:Encountered a failure at {'Batch': 1, 'Submissions': 0, 'Comments': 0}
INFO:root:Encountered a failure at {'Batch': 2, 'Submissions': 10, 'Comments': 100}
INFO:root:Encountered a failure at {'Batch': 3, 'Submissions': 30, 'Comments': 300}
INFO:root:Encountered a failure at {'Batch': 4, 'Submissions': 60, 'Comments': 600}
INFO:root:Encountered a failure at {'Batch': 5, 'Submissions': 100, 'Comments': 1000}
INFO:root:Encountered a failure at {'Batch': 6, 'Submissions': 150, 'Comments': 1500}
INFO:scraper.monitor:Exceeded failure threshold of 5 failures. Opening the circuit.


In [7]:
cb.failure(context=context)

INFO:root:Encountered a failure at {'Batch': 6, 'Submissions': 150, 'Comments': 1500}
INFO:scraper.monitor:Changing the circuit from Open to Half-Open state.


In [8]:
cb.success()

In [9]:
cb.success()
cb.success()
cb.success()
cb.success()
cb.success()

INFO:scraper.monitor:Encountered 4 successes in a row. Closing circuit.
